## GroupBy and Joins in Spark
- How groupBy works internally in Spark
- Shuffling
- Joining two large tables
- Merge Sort Join
- Joining a large table to a small table
- Broadcasting
- [Video](https://youtu.be/9qrDsY_2COo)

In [1]:
import pyspark
from pyspark.sql import SparkSession

#### Create a Local Spark Cluster

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/07/02 00:20:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/02 00:20:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/02 00:20:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/07/02 00:20:55 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


#### Load the Data and Register a Temporary Table

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/home/aayomide/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


#### Group By

In [5]:
# get the total amount and number of trips per hour, since January 2021
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [8]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [9]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [10]:
spark.read.parquet('data/report/revenue/green')

DataFrame[hour: timestamp, zone: int, amount: double, number_records: bigint]

#### Spark Joins

In [11]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [12]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [13]:
df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [14]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [25]:
df_join.show(10)

+-------------------+----+------------+--------------------+-------------+---------------------+
|               hour|zone|green_amount|green_number_records|yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+-------------+---------------------+
|2020-02-02 23:00:00|  43|        null|                null|         14.8|                    1|
|2020-02-02 23:00:00|  90|        null|                null|        17.16|                    1|
|2020-02-02 23:00:00| 107|        null|                null|        14.12|                    1|
|2020-02-03 17:00:00| 170|        null|                null|        17.76|                    1|
|2020-02-03 18:00:00|  48|        null|                null|          8.8|                    1|
|2020-02-03 20:00:00|  75|        null|                null|        17.76|                    1|
|2020-02-03 22:00:00|  48|        null|                null|        49.42|                    1|
|2020-02-04 08:00:00|  74|    

> The nulls are expected as some values present in green data are absent in yellow data and vice versa

In [16]:
# write the joined dataframe to a parquet file
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [17]:
df_join = spark.read.parquet('data/report/revenue/total')

In [18]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

#### Joining a Large Table and a Small Table

In [19]:
df_zones = spark.read.parquet('zones/')

In [20]:
spark.read.parquet('zones/').show(4)

+----------+---------+--------------------+------------+
|LocationID|  Borough|                Zone|service_zone|
+----------+---------+--------------------+------------+
|         1|      EWR|      Newark Airport|         EWR|
|         2|   Queens|         Jamaica Bay|   Boro Zone|
|         3|    Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|Manhattan|       Alphabet City| Yellow Zone|
+----------+---------+--------------------+------------+
only showing top 4 rows



In [21]:
# we didnt use join "on" here because the common column to join are named differently in the two datasets
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [26]:
df_result.show(5)

+-------------------+----+------------+--------------------+-------------+---------------------+----------+---------+------------+------------+
|               hour|zone|green_amount|green_number_records|yellow_amount|yellow_number_records|LocationID|  Borough|        Zone|service_zone|
+-------------------+----+------------+--------------------+-------------+---------------------+----------+---------+------------+------------+
|2020-02-02 23:00:00|  43|        null|                null|         14.8|                    1|        43|Manhattan|Central Park| Yellow Zone|
|2020-02-02 23:00:00|  90|        null|                null|        17.16|                    1|        90|Manhattan|    Flatiron| Yellow Zone|
|2020-02-02 23:00:00| 107|        null|                null|        14.12|                    1|       107|Manhattan|    Gramercy| Yellow Zone|
|2020-02-03 17:00:00| 170|        null|                null|        17.76|                    1|       170|Manhattan| Murray Hill| Yello

In [23]:
# drop locationID and zone since we now have the borough names
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones', mode='overwrite')

In [24]:
spark.read.parquet('tmp/revenue-zones').show(5)

+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|               hour|green_amount|green_number_records|yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|2020-02-02 23:00:00|        null|                null|         14.8|                    1|Manhattan| Yellow Zone|
|2020-02-02 23:00:00|        null|                null|        17.16|                    1|Manhattan| Yellow Zone|
|2020-02-02 23:00:00|        null|                null|        14.12|                    1|Manhattan| Yellow Zone|
|2020-02-03 17:00:00|        null|                null|        17.76|                    1|Manhattan| Yellow Zone|
|2020-02-03 18:00:00|        null|                null|          8.8|                    1|Manhattan| Yellow Zone|
+-------------------+------------+--------------------+-------------+-----------